# Count-1: patient count >= 11 or patient count = 0 (flag)

Description: the test is to check if all the patient counts are greater or equal to 11, expect for zero. Zero patient counts are flagged. 

Starting Author: Amy Jin (amy@careset.com)

Date: April 16th, 2018

https://docs.google.com/spreadsheets/d/1IYg01IpssJaWHo6KxO4_dSDgXtYNFy41S5cIHFLvlGQ/edit#gid=604789549

## Connection to Parenthood Server

In [2]:
# Packages import
import os
import sys
import numpy as np
import pandas as pd
from collections import Counter
import operator
import mysql.connector
import sshtunnel
import pureyaml

# Handle path
project_dir = !pwd  # dir of current script/notebook file
config_file = open(project_dir[0] + "/db.yaml");
config = pureyaml.load(config_file.read());

# Argument dictionary for sshtunnel
ssh_config = {
    'ssh_address_or_host': ('parenthood.set.care', 22),
    'ssh_username':        config['ssh_username'],
    'ssh_password':        config['ssh_password'],
    'remote_bind_address': ('127.0.0.1', 3306),
    'local_bind_address':  ('0.0.0.0', 3333),
}

# Argument dictionary for mysql.connector
mysql_config = {
    'user':     config['mysql_user'],
    'password': config['mysql_passwd'],
    'host':     config['mysql_host'],
    'database': 'patch',
    'port':     3333,
}

# Connect to Parenthood server
with sshtunnel.SSHTunnelForwarder(**ssh_config) as tunnel:
    print('SSH tunneling successful on port: {}'.format(tunnel.local_bind_port))
    connection = mysql.connector.connect(**mysql_config)
    cur = connection.cursor()
    print('MySQL server connected successfully!')

SSH tunneling successful on port: 3333
MySQL server connected successfully!


## Test Function

In [3]:
# --------------------------------------- Inputs: ---------------------------------------
# 1) db_name:                database name in server
# 2）table_name:             table name
# 3) col_name:               column to test
# --------------------------------------- Outputs: --------------------------------------
# 1) Test result:            PASS/FAIL
# 2) If FAIL, the test will print out the failed cnt_bene_id, i.e., the cnt_bene_id < 11 
#    and cnt_bene_id <> 0.


def count_1(db_name, table_name, col_name):

    with sshtunnel.SSHTunnelForwarder(**ssh_config) as tunnel:
        connection = mysql.connector.connect(**mysql_config)
        cur = connection.cursor()
        
        query = ('''
            SELECT {col}
            FROM {db}.{t1}
            WHERE {col} <>0 AND {col} < 11
            LIMIT 1;
        '''.format(col = col_name, db = db_name, t1 = table_name))

        cur.execute(query)
        rows = list(sum(cur.fetchall(), ()))

        if not len(rows):
            print ("Count-1 test result for {}.{} : PASS".format(db_name, table_name))
        else:
                   
            print ("Count-1 test result for {}.{} : FAIL".format(db_name, table_name) + '\n')
            print ("The following {} fail the test:".format(col_name) + '\n')
            
            # MySQL query to get bad patient counts
            query = ('''
                SELECT {col}
                FROM {db}.{t1}
                WHERE {col} <>0 AND {col} < 11;
            '''.format(col = col_name, db = db_name, t1 = table_name))

            cur.execute(query)
                
            for row in cur.fetchall():
                for i in range(0,len(row)):
                    print (str(row[i]), end=", ")
                print ('\n')
            
        cur.close()
        connection.close()            

## Test Example

In [4]:
count_1('sanofi', 'final_report4', 'cnt_bene_id')

Count-1 test result for _amy.test_data_good : PASS
